In [ ]:
import cv2
import dlib
from matplotlib import pyplot as plt
from imutils import face_utils
import pandas as pd
import numpy as np
import os

print("OpenCV version:", cv2.__version__)
print("dlib version:", dlib.__version__)

DATASET_DIR = "../data/logic-ethicist"
PREDICTOR_PATH = "../models/shape_predictor_68_face_landmarks.dat"

# Загружаем модель
predictor = dlib.shape_predictor(PREDICTOR_PATH)
detector = dlib.get_frontal_face_detector()

data = []

for filename in os.listdir(DATASET_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(DATASET_DIR, filename)

        image_array = np.fromfile(image_path, dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError("Изображение не загружено!")
        
        # Преобразуем изображение в оттенки серого
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Детектируем лицо
        faces = detector(gray)

        #Для каждого лица найдём ключевые точки
        for face in faces:
            landmarks = predictor(gray, face)

            points = {}
            for idx in list(range(0, 17)) + list(range(17, 27)) + list(range(60, 68)):

                points[f"point{idx}"] = {landmarks.part(idx).x, landmarks.part(idx).y}
                #points[f"y{idx}"] = landmarks.part(idx).y

                cv2.circle(gray, (landmarks.part(idx).x, landmarks.part(idx).y), 2, (0, 255, 0), -1)

            data.append(points)

            #Показ изображения с результатами
            cv2.imshow("Eyebrow Landmarks", gray)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

# Сохраняем в CSV
df = pd.DataFrame(data)
print(df)
df
df.to_csv("../68_face_landmarks.csv", index=False)

print("Сохранено:", df.shape[0], "фотографий")

# Отображаем результат
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.axis("off")
# plt.show()







OpenCV version: 4.11.0
dlib version: 19.22.99
      point0     point1     point2     point3     point4      point5  \
0   {24, 96}  {27, 118}  {32, 139}  {36, 159}  {178, 44}   {194, 58}   
1   {89, 29}  {110, 30}  {33, 130}  {150, 38}  {168, 46}   {185, 59}   
2  {216, 42}  {43, 246}  {276, 47}  {56, 304}  {75, 327}  {104, 343}   
3   {30, 79}  {33, 103}  {126, 39}  {147, 47}  {56, 166}   {184, 67}   
4   {98, 59}  {58, 115}  {60, 133}  {151, 63}  {168, 68}   {184, 77}   
5  {34, 102}  {122, 35}  {142, 38}  {161, 42}  {48, 180}   {59, 197}   

       point6      point7      point8      point9  ...     point25  \
0   {76, 206}   {96, 215}  {218, 115}  {213, 134}  ...   {160, 76}   
1   {200, 73}   {90, 211}  {107, 215}  {123, 211}  ...   {160, 78}   
2  {355, 135}  {165, 366}  {365, 189}  {355, 206}  ...  {237, 189}   
3   {81, 198}   {96, 207}  {112, 207}  {203, 127}  ...   {65, 159}   
4   {88, 198}   {210, 98}  {215, 111}  {130, 213}  ...   {156, 79}   
5   {73, 212}   {90, 223}  {2